In [389]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import typing

In [390]:
data = pd.read_csv('../data/Median_Incomes.csv')
data.head()


,Location,Household Type,TimeFrame,DataFormat,Data,Fips
0,Riverdale,All Households,2005,Dollars,66429.87643,208
1,Williamsbridge,All Households,2005,Dollars,54603.07992,212
2,Throgs Neck,All Households,2005,Dollars,60436.77167,210
3,Pelham Parkway,All Households,2005,Dollars,55273.71398,211
4,Morrisania,All Households,2005,Dollars,25750.6987,203


In [391]:
data['Household Type'].unique()

array(['All Households', 'Families', 'Families with Children',
       'Families without Children'], dtype=object)

In [392]:
print(data.shape)

(4160, 6)


In [393]:
data = data.dropna().copy()
data.shape

(4160, 6)

In [394]:
data['DataFormat'].unique()

array(['Dollars'], dtype=object)

In [395]:
df = data[['Household Type', 'Location', 'Data', 'TimeFrame']].copy()
df[['Household Type','Location']].astype('string')
df['Data'] = df['Data'].str.strip()
df['Data'] = pd.to_numeric(df['Data'], errors='coerce') 
df['Data'] = np.ceil(df['Data'])
df['Data'] = df['Data'].fillna(0).astype(int)
df.head()

,Household Type,Location,Data,TimeFrame
0,All Households,Riverdale,66430,2005
1,All Households,Williamsbridge,54604,2005
2,All Households,Throgs Neck,60437,2005
3,All Households,Pelham Parkway,55274,2005
4,All Households,Morrisania,25751,2005


In [396]:
df.rename(columns={"Data": "Yearly_income", "TimeFrame": "Date"}, inplace=True)
df.head()

,Household Type,Location,Yearly_income,Date
0,All Households,Riverdale,66430,2005
1,All Households,Williamsbridge,54604,2005
2,All Households,Throgs Neck,60437,2005
3,All Households,Pelham Parkway,55274,2005
4,All Households,Morrisania,25751,2005


In [397]:
df['Date'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2021], dtype=int64)

In [398]:
filtered_df = df[df['Date'] > 2012]

# Calculate the average income for each combination of Household Type and Location
average_income_by_group = (
    filtered_df.groupby(['Household Type', 'Location'])['Yearly_income']
    .mean()
    .reset_index()
    .rename(columns={'Yearly_income': 'Avg_Yearly_Income_Above_2012'})
)

# Merge the calculated averages back to the original DataFrame
df = df.merge(average_income_by_group, on=['Household Type', 'Location'], how='left')

# Display the updated DataFrame
df.head()

,Household Type,Location,Yearly_income,Date,Avg_Yearly_Income_Above_2012
0,All Households,Riverdale,66430,2005,63444.125
1,All Households,Williamsbridge,54604,2005,57027.125
2,All Households,Throgs Neck,60437,2005,61644.875
3,All Households,Pelham Parkway,55274,2005,54912.375
4,All Households,Morrisania,25751,2005,25512.625


In [399]:
df.shape

(4160, 5)

In [400]:
df.drop(columns=['Date'], inplace=True)
df.drop(columns=['Yearly_income'], inplace=True)
df.drop(columns=['Household Type'], inplace=True)
df = df.drop_duplicates(subset=['Location'], keep='first')

df.head()


,Location,Avg_Yearly_Income_Above_2012
0,Riverdale,63444.125
1,Williamsbridge,57027.125
2,Throgs Neck,61644.875
3,Pelham Parkway,54912.375
4,Morrisania,25512.625


In [401]:
df['Location'].unique()

array(['Riverdale', 'Williamsbridge', 'Throgs Neck', 'Pelham Parkway',
       'Morrisania', 'East Tremont', 'Bedford Park', 'University Heights',
       'Concourse/Highbridge', 'Unionport/Soundview', 'Mott Haven',
       'Hunts Point', 'Washington Heights', 'Manhattanville',
       'Central Harlem', 'East Harlem', 'Upper East Side',
       'Upper West Side', 'Chelsea/Clinton', 'Midtown Business District',
       'Murray Hill/Stuyvesant', 'Lower East Side',
       'Battery Park/Tribeca', 'Greenwich Village', 'Tottenville',
       'South Beach', 'St. George', 'Williamsburg/Greenpoint', 'Bushwick',
       'Bedford Stuyvesant', 'Fort Greene/Brooklyn Hts', 'Park Slope',
       'Crown Heights North', 'Brownsville', 'East New York', 'Canarsie',
       'East Flatbush', 'Crown Heights South', 'Sunset Park', 'Bay Ridge',
       'Borough Park', 'Flatbush/Midwood', 'Sheepshead Bay',
       'Bensonhurst', 'Coney Island', 'Astoria', 'Jackson Heights',
       'Flushing', 'Bayside', 'Queens Village', 

In [402]:
name_to_boro_cd = {
    # Manhattan (MXX)
    'Astoria': 'Q01',
    'Battery Park/Tribeca': 'M01',
    'Bay Ridge': 'K10',
    'Bayside': 'Q11',
    'Bedford Park': 'B07',
    'Bedford Stuyvesant': 'K03',
    'Bensonhurst': 'K11',
    'Borough Park': 'K12',
    'Brownsville': 'K16',
    'Bushwick': 'K04',
    'Canarsie': 'K18',
    'Central Harlem': 'M10',
    'Chelsea/Clinton': 'M04',
    'Concourse/Highbridge': 'B04',
    'Coney Island': 'K13',
    'Crown Heights North': 'K08',
    'Crown Heights South': 'K09',
    'East Flatbush': 'K17',
    'East Harlem': 'M11',
    'East New York': 'K05',
    'East Tremont': 'B06',
    'Elmhurst/Corona': 'Q04',
    'Flatbush/Midwood': 'K14',
    'Flushing': 'Q07',
    'Fort Greene/Brooklyn Hts': 'K02',
    'Fresh Meadows/Briarwood': 'Q08',
    'Greenwich Village': 'M02',
    'Howard Beach': 'Q10',
    'Hunts Point': 'B02',
    'Jackson Heights': 'Q03',
    'Jamaica/St. Albans': 'Q12',
    'Lower East Side': 'M03',
    'Manhattanville': 'M09',
    'Midtown Business District': 'M05',
    'Morrisania': 'B03',
    'Mott Haven': 'B01',
    'Murray Hill/Stuyvesant': 'M06',
    'Park Slope': 'K06',
    'Pelham Parkway': 'B11',
    'Queens Village': 'Q13',
    'Rego Park/Forest Hills': 'Q06',
    'Ridgewood/Glendale': 'Q05',
    'Riverdale': 'B08',
    'Sheepshead Bay': 'K15',
    'South Beach': 'S02',
    'St. George': 'S01',
    'Sunnyside/Woodside': 'Q02',
    'Sunset Park': 'K07',
    'The Rockaways': 'Q14',
    'Throgs Neck': 'B10',
    'Tottenville': 'S03',
    'Unionport/Soundview': 'B09',
    'University Heights': 'B05',
    'Upper East Side': 'M08',
    'Upper West Side': 'M07',
    'Washington Heights': 'M12',
    'Williamsbridge': 'B12',
    'Williamsburg/Greenpoint': 'K01',
    'Woodhaven': 'Q09',
    'Bronx':'Bronx', 
    'Brooklyn': 'Brooklyn',
    'Manhattan':'Manhattan',
    'Queens':'Queens',
    'Staten Island': 'Staten Island',
    'New York City':'New York City'
}


In [403]:
df['Location'] = df['Location'].map(name_to_boro_cd)


In [404]:

borough_letter_to_number = {
    'B': '2',  # Bronx
    'M': '1',  # Manhattan
    'K': '3',  # Brooklyn
    'Q': '4',  # Queens
    'S': '5'   # Staten Island
}


def convert_location(location):
    if location[0] in borough_letter_to_number:
        borough_letter = location[0]  # First character
        district_number = location[1:]  # Remaining characters (e.g., 08, 12)
        # Convert to numeric boro_cd
        return borough_letter_to_number[borough_letter] + district_number
    else:
        return location


df['Location'] = df['Location'].apply(convert_location)
print(df[['Location']])


         Location
0             208
1             212
2             210
3             211
4             203
..            ...
60       2rooklyn
61      1anhattan
62         4ueens
63  5taten Island
64  New York City

[65 rows x 1 columns]


In [405]:
df['Location'].unique()

array(['208', '212', '210', '211', '203', '206', '207', '205', '204',
       '209', '201', '202', '112', '109', '110', '111', '108', '107',
       '104', '105', '106', '103', '101', '102', '503', '502', '501',
       '301', '304', '303', '302', '306', '308', '316', '305', '318',
       '317', '309', '307', '310', '312', '314', '315', '311', '313',
       '401', '403', '407', '411', '413', '408', '404', '406', '402',
       '405', '409', '412', '410', '414', '2ronx', '2rooklyn',
       '1anhattan', '4ueens', '5taten Island', 'New York City'],
      dtype=object)

In [406]:
# Mapping for correcting borough names
borough_name_fix = {
    '2ronx': 'Bronx',
    '2rooklyn': 'Brooklyn',
    '1anhattan': 'Manhattan',
    '4ueens': 'Queens',
    '5taten Island': 'Staten Island'
}

# Apply the mapping to fix borough names
df['Location'] = df['Location'].replace(borough_name_fix)

# Display the updated DataFrame
print(df['Location'].unique())  # Check the unique values after correction


['208' '212' '210' '211' '203' '206' '207' '205' '204' '209' '201' '202'
 '112' '109' '110' '111' '108' '107' '104' '105' '106' '103' '101' '102'
 '503' '502' '501' '301' '304' '303' '302' '306' '308' '316' '305' '318'
 '317' '309' '307' '310' '312' '314' '315' '311' '313' '401' '403' '407'
 '411' '413' '408' '404' '406' '402' '405' '409' '412' '410' '414' 'Bronx'
 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'New York City']


In [407]:
df.rename(columns={"Location": "boro_cd"}, inplace=True)
df.head()

,boro_cd,Avg_Yearly_Income_Above_2012
0,208,63444.125
1,212,57027.125
2,210,61644.875
3,211,54912.375
4,203,25512.625


In [408]:

df.to_csv('../exports/Income.csv', index=False)
